# DataFrame基础 (三)

In [1]:
import numpy as np
import pandas as pd

In [2]:
np.random.seed(101)

### 具有多层标题的DataFrame

In [3]:
# 出勤率(%)抽查表
terms = '上学期 上学期 上学期 下学期 下学期 下学期'.split()
weeks = '第3周 第12周 第18周 第3周 第12周 第18周'.split()
combine = list(zip(terms,weeks)) # zip() 对应元素组合成tuple，在转为列表
multi_index = pd.MultiIndex.from_tuples(combine) # 多层行索引
multi_index

MultiIndex(levels=[['上学期', '下学期'], ['第12周', '第18周', '第3周']],
           labels=[[0, 0, 0, 1, 1, 1], [2, 0, 1, 2, 0, 1]])

In [4]:
classes = '初三1班 初三2班 初三3班'.split()
attendance_table = pd.DataFrame(np.random.randint(85,100,(6,3)),multi_index,classes) #
attendance_table

初三1班  初三2班  初三3班
上学期 第3周     96    86    91
    第12周    92    98    96
    第18周    94    98    93
下学期 第3周     89    93    90
    第12周    97    97    85
    第18周    99    90    97

In [5]:
attendance_table.index.names=['2018学年','周数'] # 给内外层 行索引起名
attendance_table

初三1班  初三2班  初三3班
2018学年 周数                    
上学期    第3周     96    86    91
       第12周    92    98    96
       第18周    94    98    93
下学期    第3周     89    93    90
       第12周    97    97    85
       第18周    99    90    97

In [6]:
attendance_table.初三1班 # 获取列

2018学年  周数  
上学期     第3周     96
        第12周    92
        第18周    94
下学期     第3周     89
        第12周    97
        第18周    99
Name: 初三1班, dtype: int64

In [7]:
attendance_table.loc['上学期'] # 获取外围index行 注意中括号 DataFrame.loc['行标题']

,初三1班,初三2班,初三3班
周数,,,
第3周,96,86,91
第12周,92,98,96
第18周,94,98,93


In [8]:
attendance_table.loc['上学期'].初三3班 # 获取外围index行 中的某列

周数
第3周     91
第12周    96
第18周    93
Name: 初三3班, dtype: int64

In [9]:
attendance_table.loc['上学期'].初三3班.loc['第3周'] # 获取外围index行 中的某列 的某行

91

In [10]:
attendance_table.xs('上学期') # 获取外层索引的行 同attendance_table.loc['上学期']

,初三1班,初三2班,初三3班
周数,,,
第3周,96,86,91
第12周,92,98,96
第18周,94,98,93


In [11]:
attendance_table.xs('第3周',level='周数') # 在该级别的行内，取出索引相同的

,初三1班,初三2班,初三3班
2018学年,,,
上学期,96,86,91
下学期,89,93,90


### 处理DataFrame中缺失的数据 (Missing Data)

In [12]:
attendance_table.dtypes # 类型为int32的，无法修改数据为nan

初三1班    int64
初三2班    int64
初三3班    int64
dtype: object

In [13]:
b = attendance_table.astype('object').loc['上学期'] # 将数据类型改为object ， 再取出上学期的表
b.dtypes # 全是object类型，可以修改数据为nan

初三1班    object
初三2班    object
初三3班    object
dtype: object

In [14]:
b.初三3班.loc['第12周']=np.nan # 模拟数据缺失
b.初三3班.loc['第18周']=np.nan # 模拟数据缺失
b.初三2班.loc['第18周']=np.nan # 模拟数据缺失
b

,初三1班,初三2班,初三3班
周数,,,
第3周,96,86,91
第12周,92,98,NaN
第18周,94,NaN,NaN


In [15]:
b.dropna() # 去掉有数据缺失的行

,初三1班,初三2班,初三3班
周数,,,
第3周,96,86,91


In [16]:
b.dropna(axis=1) # 去掉有数据缺失的列

,初三1班
周数,
第3周,96
第12周,92
第18周,94


In [17]:
b.dropna(thresh=2) # 去掉有数据缺失的行，并且缺失数据达到2个

,初三1班,初三2班,初三3班
周数,,,
第3周,96,86,91
第12周,92,98,NaN


In [18]:
b.fillna(80) # 将缺失数据 都替换为 此数值

,初三1班,初三2班,初三3班
周数,,,
第3周,96,86,91
第12周,92,98,80
第18周,94,80,80


In [19]:
b.初三2班.fillna(b.初三2班.mean(),inplace=True) # 用该列其他数据的平均值来填充 缺失数据
b

,初三1班,初三2班,初三3班
周数,,,
第3周,96,86.0,91
第12周,92,98.0,NaN
第18周,94,92.0,NaN
